In [1]:
import numpy as np
import statsmodels.api as sm

In [2]:
import pandas as pd
file = '/Users/shionguha/Documents/GitHub/inf2178h-experimentaldesign-hcds/data/hsb2.csv'
hsb2 = pd.read_table(file, delimiter=",")

In [3]:
hsb2.head(10)

,id,gender,race,ses,schtyp,prog,read,write,math,science,socst
0,70,male,white,low,public,general,57,52,41,47,57
1,121,female,white,middle,public,vocational,68,59,53,63,61
2,86,male,white,high,public,general,44,33,54,58,31
3,141,male,white,high,public,vocational,63,44,47,53,56
4,172,male,white,middle,public,academic,47,52,57,53,61
5,113,male,white,middle,public,academic,44,52,51,63,61
6,50,male,african american,middle,public,general,50,59,42,53,61
7,11,male,hispanic,middle,public,academic,34,46,45,39,36
8,84,male,white,middle,public,general,63,57,54,58,51
9,48,male,african american,middle,public,academic,57,55,52,50,51


In [4]:
hsb2.groupby('race')['write'].mean()

race
african american    48.200000
asian               58.000000
hispanic            46.458333
white               54.055172
Name: write, dtype: float64

In [5]:
#dummy coding
from patsy.contrasts import Treatment
levels = [1,2,3,4]
contrast = Treatment(reference=0).code_without_intercept(levels)
print(contrast.matrix)

[[0. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [6]:
hsb2.race.head(10)

0               white
1               white
2               white
3               white
4               white
5               white
6    african american
7            hispanic
8               white
9    african american
Name: race, dtype: object

In [8]:
pd.get_dummies(hsb2.race.values, drop_first=False)

,african american,asian,hispanic,white
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
195,0,1,0,0
196,0,0,0,1
197,0,0,0,1
198,0,0,0,1


In [9]:
from statsmodels.formula.api import ols
mod = ols("write ~ C(race, Treatment)", data=hsb2)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  write   R-squared:                       0.107
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     7.833
Date:                Mon, 15 Mar 2021   Prob (F-statistic):           5.78e-05
Time:                        12:02:31   Log-Likelihood:                -721.77
No. Observations:                 200   AIC:                             1452.
Df Residuals:                     196   BIC:                             1465.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   

In [10]:
#simple coding
from patsy.contrasts import ContrastMatrix

def _name_levels(prefix, levels):
    return ["[%s%s]" % (prefix, level) for level in levels]

class Simple(object):
    def _simple_contrast(self, levels):
        nlevels = len(levels)
        contr = -1./nlevels * np.ones((nlevels, nlevels-1))
        contr[1:][np.diag_indices(nlevels-1)] = (nlevels-1.)/nlevels
        return contr

    def code_with_intercept(self, levels):
        contrast = np.column_stack((np.ones(len(levels)),
                                    self._simple_contrast(levels)))
        return ContrastMatrix(contrast, _name_levels("Simp.", levels))

    def code_without_intercept(self, levels):
        contrast = self._simple_contrast(levels)
        return ContrastMatrix(contrast, _name_levels("Simp.", levels[:-1]))

In [11]:
hsb2.groupby('race')['write'].mean().mean()

51.678376436781605

In [12]:
contrast = Simple().code_without_intercept(levels)
print(contrast.matrix)

[[-0.25 -0.25 -0.25]
 [ 0.75 -0.25 -0.25]
 [-0.25  0.75 -0.25]
 [-0.25 -0.25  0.75]]


In [13]:
mod = ols("write ~ C(race, Simple)", data=hsb2)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  write   R-squared:                       0.107
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     7.833
Date:                Mon, 15 Mar 2021   Prob (F-statistic):           5.78e-05
Time:                        12:03:05   Log-Likelihood:                -721.77
No. Observations:                 200   AIC:                             1452.
Df Residuals:                     196   BIC:                             1465.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

In [14]:
#sum/deviation coding
from patsy.contrasts import Sum
contrast = Sum().code_without_intercept(levels)
print(contrast.matrix)

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [-1. -1. -1.]]


In [15]:
mod = ols("write ~ C(race, Sum)", data=hsb2)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  write   R-squared:                       0.107
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     7.833
Date:                Mon, 15 Mar 2021   Prob (F-statistic):           5.78e-05
Time:                        12:03:27   Log-Likelihood:                -721.77
No. Observations:                 200   AIC:                             1452.
Df Residuals:                     196   BIC:                             1465.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

In [16]:
hsb2.groupby('race')['write'].mean().mean()

51.678376436781605

In [17]:
#backward difference coding
from patsy.contrasts import Diff
contrast = Diff().code_without_intercept(levels)
print(contrast.matrix)

[[-0.75 -0.5  -0.25]
 [ 0.25 -0.5  -0.25]
 [ 0.25  0.5  -0.25]
 [ 0.25  0.5   0.75]]


In [18]:
mod = ols("write ~ C(race, Diff)", data=hsb2)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  write   R-squared:                       0.107
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     7.833
Date:                Mon, 15 Mar 2021   Prob (F-statistic):           5.78e-05
Time:                        12:03:56   Log-Likelihood:                -721.77
No. Observations:                 200   AIC:                             1452.
Df Residuals:                     196   BIC:                             1465.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Interc

In [20]:
#helmert coding
from patsy.contrasts import Helmert
contrast = Helmert().code_without_intercept(levels)
print(contrast.matrix)

[[-1. -1. -1.]
 [ 1. -1. -1.]
 [ 0.  2. -1.]
 [ 0.  0.  3.]]


In [22]:
mod = ols("write ~ C(race, Helmert)", data=hsb2)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  write   R-squared:                       0.107
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     7.833
Date:                Mon, 15 Mar 2021   Prob (F-statistic):           5.78e-05
Time:                        12:05:24   Log-Likelihood:                -721.77
No. Observations:                 200   AIC:                             1452.
Df Residuals:                     196   BIC:                             1465.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

In [24]:
#orthogonal polynomial coding
hsb2['readcat'] = np.asarray(pd.cut(hsb2.read, bins=3))
hsb2.groupby('readcat').mean()['write']

readcat
(27.952, 44.0]    45.000000
(44.0, 60.0]      53.356436
(60.0, 76.0]      60.127660
Name: write, dtype: float64

In [25]:
from patsy.contrasts import Poly
levels = hsb2.readcat.unique().tolist()
contrast = Poly().code_without_intercept(levels)
print(contrast.matrix)

[[-7.07106781e-01  4.08248290e-01]
 [-4.43378006e-17 -8.16496581e-01]
 [ 7.07106781e-01  4.08248290e-01]]


In [26]:
mod = ols("write ~ C(readcat, Poly)", data=hsb2)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  write   R-squared:                       0.320
Model:                            OLS   Adj. R-squared:                  0.313
Method:                 Least Squares   F-statistic:                     46.32
Date:                Mon, 15 Mar 2021   Prob (F-statistic):           3.25e-17
Time:                        12:06:10   Log-Likelihood:                -694.55
No. Observations:                 200   AIC:                             1395.
Df Residuals:                     197   BIC:                             1405.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           